In [325]:
import pandas as pd
import sqlite3

In [326]:
conn = sqlite3.connect('ECDC_Venue_Cases.db')
cur = conn.cursor()

In [327]:
#Dimension: Quarantine

cur.execute("DROP TABLE IF EXISTS QuarantineDT")
cur.execute('''CREATE TABLE QuarantineDT(
quarantineId INTEGER PRIMARY KEY,
quarantineName TEXT,
numberOfDays INTEGER
) ''')

In [328]:
#Dimension: Vaccine

cur.execute("DROP TABLE IF EXISTS VaccineDT")
cur.execute('''CREATE TABLE VaccineDT(
vaccineId INTEGER PRIMARY KEY NOT NULL,
vaccineName TEXT
) ''')

In [329]:
#Dimension: Venue

cur.execute("DROP TABLE IF EXISTS VenueDT")
cur.execute('''CREATE TABLE VenueDT(
venueId INTEGER PRIMARY KEY NOT NULL,
venueName TEXT,
venueType TEXT,
venueAddress TEXT
) ''')

In [330]:
#Dimension: DiseaseVariant

cur.execute("DROP TABLE IF EXISTS DiseaseVariantDT")
cur.execute('''CREATE TABLE DiseaseVariantDT(
diseaseVariantId INTEGER PRIMARY KEY NOT NULL,
diseaseId TEXT,
variantName TEXT
) ''')

In [331]:
#Dimension: Drug

cur.execute("DROP TABLE IF EXISTS DrugDT")
cur.execute('''CREATE TABLE DrugDT(
drugId INTEGER PRIMARY KEY,
drugName TEXT,
forDisease TEXT
) ''')

In [332]:
#Dimension: PatientStatus

cur.execute("DROP TABLE IF EXISTS PatientStatusDT")
cur.execute('''CREATE TABLE PatientStatusDT(
patientStatusId INTEGER PRIMARY KEY NOT NULL,
patientStatusName TEXT
) ''')

In [333]:
#Dimension: HealthUnit

cur.execute("DROP TABLE IF EXISTS HealthUnitDT")
cur.execute('''CREATE TABLE HealthUnitDT(
healthUnitId INTEGER PRIMARY KEY,
healthUnitName TEXT,
address TEXT,
districtId TEXT
) ''')

In [334]:
#Dimension: Time

cur.execute("DROP TABLE IF EXISTS TimeDT")
cur.execute('''CREATE TABLE TimeDT(
dateId INTEGER PRIMARY KEY NOT NULL,
startDate TEXT,
endDate TEXT,
yearId INTEGER,
monthId INTEGER,
weekId INTEGER,
dayOfWeekId INTEGER
) ''')

In [335]:
#Dimension: Citizen

cur.execute("DROP TABLE IF EXISTS CitizenDT")
cur.execute('''CREATE TABLE CitizenDT(
citizenId INTEGER PRIMARY KEY NOT NULL,
name TEXT,
surname TEXT,
gender TEXT,
dateOfBirth TEXT,
weight INTEGER,
height INTEGER,
phoneNumber TEXT,
address TEXT,
districtId TEXT
) ''')

In [336]:
#Dimension: DiseaseCase

cur.execute("DROP TABLE IF EXISTS DiseaseCaseDT")
cur.execute('''CREATE TABLE DiseaseCaseDT(
patientId INTEGER PRIMARY KEY NOT NULL,
citizenId INTEGER NOT NULL,
patientStatusId INTEGER,
diseaseVariantId INTEGER,
FOREIGN KEY (citizenId) REFERENCES CitizenDT(citizenId),
FOREIGN KEY (patientStatusId) REFERENCES PatientStatusDT(patientStatusId),
FOREIGN KEY (diseaseVariantId) REFERENCES DiseaseVariantDT(diseaseVariantId)
) ''')

In [337]:
#Fact: VenueCases

cur.execute("DROP TABLE IF EXISTS VenueCasesFT")
cur.execute('''CREATE TABLE VenueCasesFT(
venueId INTEGER NOT NULL,
dateId INTEGER NOT NULL,
patientId INTEGER,
vaccineId INTEGER,
quarantineId INTEGER,
drugId INTEGER,
FOREIGN KEY (venueId) REFERENCES VenueDT(venueId),
FOREIGN KEY (dateId) REFERENCES TimeDT(dateId),
FOREIGN KEY (patientId) REFERENCES DiseaseCaseDT(patientId)
FOREIGN KEY (vaccineId) REFERENCES VaccineDT(vaccineId),
FOREIGN KEY (quarantineId) REFERENCES QuarantineDT(quarantineId),
FOREIGN KEY (drugId) REFERENCES DrugDT(drugId)
) ''')

In [338]:
#example data insertion and successful integration:

quarantine_data = [(1, "Quarantine Facility 1", 30),(2, "Quarantine Facility 2", 15)]
vaccine_data = [(1, "Moderna"),(2, "Pfizer-BioNTech")]
venue_data = [(1, "ECDC Hospital 1", "Hospital", "77 Bob Avenue SE7 3MV"), (2, "ECDC Hospital 2", "Hospital", "66 John Avenue SE8 4MV")]
disease_variant_data = [(1, "COVID-19", "Alpha"), (2, "COVID-19", "Beta")]
drug_data = [(1, "Remdesivir", "COVID-19"), (2, "Dexamethasone", "COVID-19")]
patient_status_data = [(1, "Quarantined"), (2, "Recovered")]
health_unit_data = [(1, 'Health Unit 1', '77 Tree St', 'District A'), (2 , 'Health Unit 2', '55 Leaf St', 'District B' )]
time_data = [(1, "10-02-2024", "11-03-2024", 2024, 2, 2, 6 ), (2, "4-05-2023", "19-05-2023", 2024, 5, 1, 7)]
citizen_data = [(101, "Franklin", "Smith", "Male", "1-05-1995", 60, 180, "07851 857667", "69 Ferror Avenue S5 9MS", "District A"), (133, "Tarik", "Fred", "Female", "5/07/1990", 55, 170, "07251 852637", "96 White Avenue N5 3MS", "District C")]
disease_case_data = [(1,101,1,1), (2,133,2,2)]
venue_cases_data = [(1,1,101,2,1,2), (2,2,133,1,2,1)]

cur.executemany('INSERT INTO QuarantineDT VALUES (?,?,?)', quarantine_data)
cur.executemany('INSERT INTO VaccineDT VALUES (?,?)', vaccine_data)
cur.executemany('INSERT INTO VenueDT VALUES (?,?,?,?)', venue_data)
cur.executemany('INSERT INTO DiseaseVariantDT VALUES (?,?,?)', disease_variant_data)
cur.executemany('INSERT INTO DrugDT VALUES (?,?,?)', drug_data)
cur.executemany('INSERT INTO PatientStatusDT VALUES (?,?)', patient_status_data)
cur.executemany('INSERT INTO HealthUnitDT VALUES (?,?,?,?)', health_unit_data)
cur.executemany('INSERT INTO TimeDT VALUES (?,?,?,?,?,?,?)', time_data)
cur.executemany('INSERT INTO CitizenDT VALUES (?,?,?,?,?,?,?,?,?,?)', citizen_data)
cur.executemany('INSERT INTO DiseaseCaseDT VALUES (?,?,?,?)', disease_case_data)
cur.executemany('INSERT INTO VenueCasesFT VALUES (?,?,?,?,?,?)', venue_cases_data)

#conn.commit()
#conn.close()

In [339]:
cur.execute("DROP VIEW IF EXISTS m1")
cur.execute ('''CREATE VIEW m1 AS
                SELECT f.venueId, t.monthId AS month, f.patientId,
                COUNT(f.vaccineId) /COUNT(*) AS vaccinationRate, COUNT(f.quarantineId) AS quarantineNumbers,
                COUNT(f.drugId) AS drugTreatmentNumbers
                FROM VenueCasesFT f INNER JOIN TimeDT t ON f.dateId = t.dateId
                GROUP BY f.venueId, month
                ''')

In [340]:
M1 = pd.read_sql('SELECT * FROM m1', conn)
M1

,venueId,month,patientId,vaccinationRate,quarantineNumbers,drugTreatmentNumbers
0,1,2,101,1,1,1
1,2,5,133,1,1,1
